In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Feature extraction from skin images

In [ ]:
from os import listdir
from PIL import Image as PImage
import numpy as np
import pandas as pd
import cv2
from sklearn.model_selection import train_test_split
from keras.layers import Conv2D,MaxPooling2D,Flatten
from keras.layers import Dense, GlobalAveragePooling2D,Dropout,Input
# from keras.layers.advanced_activations import LeakyReLU, ELU
from keras.models import Sequential,Model

import matplotlib.pyplot as plt

from keras.preprocessing.image import load_img

from sklearn.model_selection import StratifiedKFold

from keras_preprocessing.image import ImageDataGenerator

from keras.preprocessing.image import img_to_array

from numpy import expand_dims

from matplotlib import pyplot

#ISIC2018 imageISIC2018
path = "/content/drive/My Drive/app/skin dataset/ISIC2018_Task3_Training_Input/"

#GroundTruth

labeldf = pd.read_csv("/content/drive/My Drive/app/skin dataset/ISIC2018_Task3_Training_GroundTruth.csv")

imagesList = listdir(path)

images = []

labels=[]

counter=1

label1counter=0

for image in imagesList:

    rowimg = load_img(path+image,target_size=(256,256))

    data= img_to_array(rowimg)

    key=image.split(".")[0]

    cls = np.where(labeldf.query('image==@key').values[0, 1:])[0]

    clslabel=cls[0]

    images.append(data.astype('uint8'))

    labels.append(clslabel)

    counter=counter+1

images = np.array(images,dtype = 'float16')

labels = np.array(labels,dtype = 'float16')


Feature extraction from skin images and data augmentation for balancing classes using Keras ImageDataGenerator

In [ ]:
from os import listdir
from PIL import Image as PImage
import numpy as np
import pandas as pd
import cv2
from sklearn.model_selection import train_test_split
from keras.layers import Conv2D,MaxPooling2D,Flatten
from keras.layers import Dense, GlobalAveragePooling2D,Dropout,Input
# from keras.layers.advanced_activations import LeakyReLU, ELU
from keras.models import Sequential,Model

from keras.layers.normalization import BatchNormalization

from keras.utils import to_categorical

import matplotlib.pyplot as plt

from keras.preprocessing.image import load_img

from sklearn.model_selection import StratifiedKFold

from keras_preprocessing.image import ImageDataGenerator

from keras.preprocessing.image import img_to_array

from numpy import expand_dims

from matplotlib import pyplot


path = "/content/drive/My Drive/app/skin dataset/ISIC2018_Task3_Training_Input/"

labeldf = pd.read_csv("/content/drive/My Drive/app/skin dataset/ISIC2018_Task3_Training_GroundTruth.csv")


imagesList = listdir(path)


images = []

labels=[]

f0=0
f1=0
f2=0
f3=0
f4=0
f5=0
f6=0

datagen = ImageDataGenerator(
        rotation_range=180,
        width_shift_range=0.1,
        height_shift_range=0.1,
        zoom_range=0.1,
        horizontal_flip=True,
        vertical_flip=True,
        #brightness_range=(0.9,1.1),
        fill_mode='nearest')


counter=1

label1counter=0

for image in imagesList:


    rowimg = load_img(path+image,target_size=(128,128))

    data= img_to_array(rowimg)

    key=image.split(".")[0]

    cls = np.where(labeldf.query('image==@key').values[0, 1:])[0]

    clslabel=cls[0]

    images.append(data.astype('uint8'))

    labels.append(clslabel)

    lenght=0


    # MEL / 0 ==>1113
    # NV  / 1 ==>6705
    # BCC  / 2 ==>514
    # AKIES  / 3 ==>327
    # BKL  / 4  ==>1099
    # DF  / 5 ==>115
    # VASC  / 6 ==>142

    if clslabel==1 :

       f1=f1+1
       lenght=0

    if clslabel==0 :

       lenght=5

       f0=f0+1+0


    if clslabel==4 :

       lenght=5

       f4=f4+1+0

    if clslabel==2 :

       lenght=19

       f2=f2+1+19

    if clslabel==3 :

       lenght=17

       f3=f3+1+8


    if clslabel==6 :

       lenght=41

       f6=f6+1+9


    if clslabel==5 :

       lenght=42

       f5=f5+1+25


    for i in range(lenght):


        samples = expand_dims(data, 0)


        it = datagen.flow(samples , batch_size=1)


        batch =it.next()


        img2 =batch[0].astype('uint8')



        images.append(img2)


        labels.append(clslabel)


    counter=counter+1


images = np.array(images,dtype = 'float16')

labels = np.array(labels,dtype = 'float16')



Genetic algorithm for finding best architecture and hyper-parameter optimization of scratch models

In [ ]:
import random
import numpy
from keras.layers import Conv2D,MaxPooling2D,Flatten
from keras.layers import Dense, GlobalAveragePooling2D,Dropout,Input
from keras.models import Sequential,Model
from keras.layers.normalization import BatchNormalization
from keras.callbacks import EarlyStopping, ModelCheckpoint
import gc


pop_size=10

batch_size=32

numconvfilterslist = [2,4,8,16,32,64,128,256]

numconvfiltersizelist  = [2,3,4,5,6,7,8,9]

numpoolingsizelist  = [0,2,3,4,5,6,7,8,9]

num_dense_nerons_list = [2,4,8,16,32,64,128,256,512,1024,2048,4096]


def findmaximum(fitness2, num_parents):

    maxarrayindces=[]

    for parent_num in range(num_parents):

        max_fitness_idx = numpy.where(fitness2 == numpy.max(fitness2))

        max_fitness_idx = max_fitness_idx[0][0]

        fitness2[max_fitness_idx] = -99999999999

        maxarrayindces.append(max_fitness_idx)

    return maxarrayindces


def select_mating_pool(pop, fitness, num_parents):

    # Selecting the best individuals in the current generation as parents for producing the offspring of the next generation.

    parents = numpy.empty((num_parents, pop.shape[1]))

    parents=parents.astype("int32")

    for parent_num in range(num_parents):

        max_fitness_idx = numpy.where(fitness == numpy.max(fitness))

        max_fitness_idx = max_fitness_idx[0][0]

        #print('max_fitness_idx=>',max_fitness_idx)

        parents[parent_num, :] = pop[max_fitness_idx, :]

        fitness[max_fitness_idx] = -99999999999

    return parents

def createDeeepModel(convlayernum,
                     convlayer_1_filternum,convlayer_1_size,pooling_1_size,
                     convlayer_2_filternum,convlayer_2_size,pooling_2_size,
                     convlayer_3_filternum,convlayer_3_size,pooling_3_size,
                     convlayer_4_filternum,convlayer_4_size,pooling_4_size,
                     convlayer_5_filternum,convlayer_5_size,pooling_5_size,
                     convlayer_6_filternum,convlayer_6_size,pooling_6_size,
                     convlayer_7_filternum,convlayer_7_size,pooling_7_size,
                     convlayer_8_filternum,convlayer_8_size,pooling_8_size,
                     convlayer_9_filternum,convlayer_9_size,pooling_9_size,
                     convlayer_10_filternum,convlayer_10_size,pooling_10_size,
                     convlayer_11_filternum,convlayer_11_size,pooling_11_size,
                     convlayer_12_filternum,convlayer_12_size,pooling_12_size,
                     convlayer_13_filternum,convlayer_13_size,pooling_13_size,
                     denselayernum,
                     dense_1_nerouns,dense_2_nerouns,dense_3_nerouns):

    model = Sequential()

    inputdim=256

    #onelayer

    if convlayer_1_filternum !=0 and convlayer_2_filternum==0 :


       model.add(Conv2D(filters = convlayer_1_filternum, kernel_size = convlayer_1_size, padding = 'same', activation = 'relu', input_shape = (inputdim,inputdim,3)))

       if pooling_1_size !=0 :


          model.add(MaxPooling2D(pool_size=pooling_1_size,padding='same'))


    #2 layer
    elif  convlayer_1_filternum !=0 and convlayer_2_filternum !=0 and convlayer_3_filternum==0:

       model.add(Conv2D(filters = convlayer_1_filternum, kernel_size = convlayer_1_size, padding = 'same', activation = 'relu', input_shape = (inputdim,inputdim,3)))

       if pooling_1_size !=0 :

          model.add(MaxPooling2D(pool_size=pooling_1_size,padding='same'))


       model.add(Conv2D(filters = convlayer_2_filternum, kernel_size = convlayer_2_size, padding = 'same', activation = 'relu'))

       if pooling_2_size !=0 :

          model.add(MaxPooling2D(pool_size=pooling_2_size,padding='same'))

    #3 layer
    elif  convlayer_1_filternum !=0 and convlayer_2_filternum !=0 and convlayer_3_filternum!=0 and  convlayer_4_filternum==0:


       model.add(Conv2D(filters = convlayer_1_filternum, kernel_size = convlayer_1_size, padding = 'same', activation = 'relu', input_shape = (inputdim,inputdim,3)))

       if pooling_1_size !=0 :

          model.add(MaxPooling2D(pool_size=pooling_1_size,padding='same'))


       model.add(Conv2D(filters = convlayer_2_filternum, kernel_size = convlayer_2_size, padding = 'same', activation = 'relu'))

       if pooling_2_size !=0 :

          model.add(MaxPooling2D(pool_size=pooling_2_size,padding='same'))



       model.add(Conv2D(filters = convlayer_3_filternum, kernel_size = convlayer_3_size, padding = 'same', activation = 'relu'))

       if pooling_3_size !=0 :

          model.add(MaxPooling2D(pool_size=pooling_3_size,padding='same'))


    #4 layer
    elif  convlayer_1_filternum !=0 and convlayer_2_filternum !=0 and convlayer_3_filternum!=0 and  convlayer_4_filternum!=0 and  convlayer_5_filternum==0 :


       model.add(Conv2D(filters = convlayer_1_filternum, kernel_size = convlayer_1_size, padding = 'same', activation = 'relu', input_shape = (inputdim,inputdim,3)))

       if pooling_1_size !=0 :

          model.add(MaxPooling2D(pool_size=pooling_1_size,padding='same'))


       model.add(Conv2D(filters = convlayer_2_filternum, kernel_size = convlayer_2_size, padding = 'same', activation = 'relu'))

       if pooling_2_size !=0 :

          model.add(MaxPooling2D(pool_size=pooling_2_size,padding='same'))



       model.add(Conv2D(filters = convlayer_3_filternum, kernel_size = convlayer_3_size, padding = 'same', activation = 'relu'))

       if pooling_3_size !=0 :

          model.add(MaxPooling2D(pool_size=pooling_3_size,padding='same'))


       model.add(Conv2D(filters = convlayer_4_filternum, kernel_size = convlayer_4_size, padding = 'same', activation = 'relu'))

       if pooling_4_size !=0 :

          model.add(MaxPooling2D(pool_size=pooling_4_size,padding='same'))



    #5 layer
    elif  convlayer_1_filternum !=0 and convlayer_2_filternum !=0 and convlayer_3_filternum!=0 and  convlayer_4_filternum!=0 and  convlayer_5_filternum!=0 and convlayer_6_filternum ==0  :

       model.add(Conv2D(filters = convlayer_1_filternum, kernel_size = convlayer_1_size, padding = 'same', activation = 'relu', input_shape = (inputdim,inputdim,3)))

       if pooling_1_size !=0 :

          model.add(MaxPooling2D(pool_size=pooling_1_size,padding='same'))


       model.add(Conv2D(filters = convlayer_2_filternum, kernel_size = convlayer_2_size, padding = 'same', activation = 'relu'))

       if pooling_2_size !=0 :

          model.add(MaxPooling2D(pool_size=pooling_2_size,padding='same'))



       model.add(Conv2D(filters = convlayer_3_filternum, kernel_size = convlayer_3_size, padding = 'same', activation = 'relu'))

       if pooling_3_size !=0 :

          model.add(MaxPooling2D(pool_size=pooling_3_size,padding='same'))


       model.add(Conv2D(filters = convlayer_4_filternum, kernel_size = convlayer_4_size, padding = 'same', activation = 'relu'))

       if pooling_4_size !=0 :

          model.add(MaxPooling2D(pool_size=pooling_4_size,padding='same'))

       model.add(Conv2D(filters = convlayer_5_filternum, kernel_size = convlayer_5_size, padding = 'same', activation = 'relu'))

       if pooling_5_size !=0 :

          model.add(MaxPooling2D(pool_size=pooling_5_size,padding='same'))

    #6 layer
    elif  convlayer_1_filternum !=0 and convlayer_2_filternum !=0 and convlayer_3_filternum!=0 and  convlayer_4_filternum!=0 and  convlayer_5_filternum!=0 and convlayer_6_filternum !=0 and convlayer_7_filternum ==0 :

       print("to black 6");
       model.add(Conv2D(filters = convlayer_1_filternum, kernel_size = convlayer_1_size, padding = 'same', activation = 'relu', input_shape = (inputdim,inputdim,3)))

       if pooling_1_size !=0 :

          model.add(MaxPooling2D(pool_size=pooling_1_size,padding='same'))


       model.add(Conv2D(filters = convlayer_2_filternum, kernel_size = convlayer_2_size, padding = 'same', activation = 'relu'))

       if pooling_2_size !=0 :

          model.add(MaxPooling2D(pool_size=pooling_2_size,padding='same'))



       model.add(Conv2D(filters = convlayer_3_filternum, kernel_size = convlayer_3_size, padding = 'same', activation = 'relu'))

       if pooling_3_size !=0 :

          model.add(MaxPooling2D(pool_size=pooling_3_size,padding='same'))


       model.add(Conv2D(filters = convlayer_4_filternum, kernel_size = convlayer_4_size, padding = 'same', activation = 'relu'))

       if pooling_4_size !=0 :

          model.add(MaxPooling2D(pool_size=pooling_4_size,padding='same'))

       model.add(Conv2D(filters = convlayer_5_filternum, kernel_size = convlayer_5_size, padding = 'same', activation = 'relu'))

       if pooling_5_size !=0 :

          model.add(MaxPooling2D(pool_size=pooling_5_size,padding='same'))

       model.add(Conv2D(filters = convlayer_6_filternum, kernel_size = convlayer_6_size, padding = 'same', activation = 'relu'))

       if pooling_6_size !=0 :

          model.add(MaxPooling2D(pool_size=pooling_6_size,padding='same'))

    #7 layer
    elif  convlayer_1_filternum !=0 and convlayer_2_filternum !=0 and convlayer_3_filternum!=0 and  convlayer_4_filternum!=0 and  convlayer_5_filternum!=0 and convlayer_6_filternum !=0 and convlayer_7_filternum !=0 and convlayer_8_filternum ==0:

       print("to black 7");
       model.add(Conv2D(filters = convlayer_1_filternum, kernel_size = convlayer_1_size, padding = 'same', activation = 'relu', input_shape = (inputdim,inputdim,3)))

       if pooling_1_size !=0 :

          model.add(MaxPooling2D(pool_size=pooling_1_size,padding='same'))


       model.add(Conv2D(filters = convlayer_2_filternum, kernel_size = convlayer_2_size, padding = 'same', activation = 'relu'))

       if pooling_2_size !=0 :

          model.add(MaxPooling2D(pool_size=pooling_2_size,padding='same'))



       model.add(Conv2D(filters = convlayer_3_filternum, kernel_size = convlayer_3_size, padding = 'same', activation = 'relu'))

       if pooling_3_size !=0 :

          model.add(MaxPooling2D(pool_size=pooling_3_size,padding='same'))


       model.add(Conv2D(filters = convlayer_4_filternum, kernel_size = convlayer_4_size, padding = 'same', activation = 'relu'))

       if pooling_4_size !=0 :

          model.add(MaxPooling2D(pool_size=pooling_4_size,padding='same'))

       model.add(Conv2D(filters = convlayer_5_filternum, kernel_size = convlayer_5_size, padding = 'same', activation = 'relu'))

       if pooling_5_size !=0 :

          model.add(MaxPooling2D(pool_size=pooling_5_size,padding='same'))

       model.add(Conv2D(filters = convlayer_6_filternum, kernel_size = convlayer_6_size, padding = 'same', activation = 'relu'))

       if pooling_6_size !=0 :

          model.add(MaxPooling2D(pool_size=pooling_6_size,padding='same'))

       model.add(Conv2D(filters = convlayer_7_filternum, kernel_size = convlayer_7_size, padding = 'same', activation = 'relu'))

       if pooling_7_size !=0 :

          model.add(MaxPooling2D(pool_size=pooling_7_size,padding='same'))

    #8 layer
    elif  convlayer_1_filternum !=0 and convlayer_2_filternum !=0 and convlayer_3_filternum!=0 and  convlayer_4_filternum!=0 and  convlayer_5_filternum!=0 and convlayer_6_filternum !=0 and convlayer_7_filternum !=0 and convlayer_8_filternum !=0 and convlayer_9_filternum ==0 :

       print("to black 8");
       model.add(Conv2D(filters = convlayer_1_filternum, kernel_size = convlayer_1_size, padding = 'same', activation = 'relu', input_shape = (inputdim,inputdim,3)))

       if pooling_1_size !=0 :

          model.add(MaxPooling2D(pool_size=pooling_1_size,padding='same'))


       model.add(Conv2D(filters = convlayer_2_filternum, kernel_size = convlayer_2_size, padding = 'same', activation = 'relu'))

       if pooling_2_size !=0 :

          model.add(MaxPooling2D(pool_size=pooling_2_size,padding='same'))



       model.add(Conv2D(filters = convlayer_3_filternum, kernel_size = convlayer_3_size, padding = 'same', activation = 'relu'))

       if pooling_3_size !=0 :

          model.add(MaxPooling2D(pool_size=pooling_3_size,padding='same'))


       model.add(Conv2D(filters = convlayer_4_filternum, kernel_size = convlayer_4_size, padding = 'same', activation = 'relu'))

       if pooling_4_size !=0 :

          model.add(MaxPooling2D(pool_size=pooling_4_size,padding='same'))

       model.add(Conv2D(filters = convlayer_5_filternum, kernel_size = convlayer_5_size, padding = 'same', activation = 'relu'))

       if pooling_5_size !=0 :

          model.add(MaxPooling2D(pool_size=pooling_5_size,padding='same'))

       model.add(Conv2D(filters = convlayer_6_filternum, kernel_size = convlayer_6_size, padding = 'same', activation = 'relu'))

       if pooling_6_size !=0 :

          model.add(MaxPooling2D(pool_size=pooling_6_size,padding='same'))

       model.add(Conv2D(filters = convlayer_7_filternum, kernel_size = convlayer_7_size, padding = 'same', activation = 'relu'))

       if pooling_7_size !=0 :

          model.add(MaxPooling2D(pool_size=pooling_7_size,padding='same'))

       model.add(Conv2D(filters = convlayer_8_filternum, kernel_size = convlayer_8_size, padding = 'same', activation = 'relu'))

       if pooling_8_size !=0 :

          model.add(MaxPooling2D(pool_size=pooling_8_size,padding='same'))

     #9 layer
    elif  convlayer_1_filternum !=0 and convlayer_2_filternum !=0 and convlayer_3_filternum!=0 and  convlayer_4_filternum!=0 and  convlayer_5_filternum!=0 and convlayer_6_filternum !=0 and convlayer_7_filternum !=0 and convlayer_8_filternum !=0 and convlayer_9_filternum !=0 and convlayer_10_filternum ==0 :

       print("to black 9");
       model.add(Conv2D(filters = convlayer_1_filternum, kernel_size = convlayer_1_size, padding = 'same', activation = 'relu', input_shape = (inputdim,inputdim,3)))

       if pooling_1_size !=0 :

          model.add(MaxPooling2D(pool_size=pooling_1_size,padding='same'))


       model.add(Conv2D(filters = convlayer_2_filternum, kernel_size = convlayer_2_size, padding = 'same', activation = 'relu'))

       if pooling_2_size !=0 :

          model.add(MaxPooling2D(pool_size=pooling_2_size,padding='same'))



       model.add(Conv2D(filters = convlayer_3_filternum, kernel_size = convlayer_3_size, padding = 'same', activation = 'relu'))

       if pooling_3_size !=0 :

          model.add(MaxPooling2D(pool_size=pooling_3_size,padding='same'))


       model.add(Conv2D(filters = convlayer_4_filternum, kernel_size = convlayer_4_size, padding = 'same', activation = 'relu'))

       if pooling_4_size !=0 :

          model.add(MaxPooling2D(pool_size=pooling_4_size,padding='same'))

       model.add(Conv2D(filters = convlayer_5_filternum, kernel_size = convlayer_5_size, padding = 'same', activation = 'relu'))

       if pooling_5_size !=0 :

          model.add(MaxPooling2D(pool_size=pooling_5_size,padding='same'))

       model.add(Conv2D(filters = convlayer_6_filternum, kernel_size = convlayer_6_size, padding = 'same', activation = 'relu'))

       if pooling_6_size !=0 :

          model.add(MaxPooling2D(pool_size=pooling_6_size,padding='same'))

       model.add(Conv2D(filters = convlayer_7_filternum, kernel_size = convlayer_7_size, padding = 'same', activation = 'relu'))

       if pooling_7_size !=0 :

          model.add(MaxPooling2D(pool_size=pooling_7_size,padding='same'))

       model.add(Conv2D(filters = convlayer_8_filternum, kernel_size = convlayer_8_size, padding = 'same', activation = 'relu'))

       if pooling_8_size !=0 :

          model.add(MaxPooling2D(pool_size=pooling_8_size,padding='same'))

       model.add(Conv2D(filters = convlayer_9_filternum, kernel_size = convlayer_9_size, padding = 'same', activation = 'relu'))

       if pooling_9_size !=0 :

          model.add(MaxPooling2D(pool_size=pooling_9_size,padding='same'))


     #10 layer
    elif  convlayer_1_filternum !=0 and convlayer_2_filternum !=0 and convlayer_3_filternum!=0 and  convlayer_4_filternum!=0 and  convlayer_5_filternum!=0 and convlayer_6_filternum !=0 and convlayer_7_filternum !=0 and convlayer_8_filternum !=0 and convlayer_9_filternum !=0 and convlayer_10_filternum !=0 and convlayer_11_filternum==0  :

       print("to black 10");
       model.add(Conv2D(filters = convlayer_1_filternum, kernel_size = convlayer_1_size, padding = 'same', activation = 'relu', input_shape = (inputdim,inputdim,3)))

       if pooling_1_size !=0 :

          model.add(MaxPooling2D(pool_size=pooling_1_size,padding='same'))


       model.add(Conv2D(filters = convlayer_2_filternum, kernel_size = convlayer_2_size, padding = 'same', activation = 'relu'))

       if pooling_2_size !=0 :

          model.add(MaxPooling2D(pool_size=pooling_2_size,padding='same'))



       model.add(Conv2D(filters = convlayer_3_filternum, kernel_size = convlayer_3_size, padding = 'same', activation = 'relu'))

       if pooling_3_size !=0 :

          model.add(MaxPooling2D(pool_size=pooling_3_size,padding='same'))


       model.add(Conv2D(filters = convlayer_4_filternum, kernel_size = convlayer_4_size, padding = 'same', activation = 'relu'))

       if pooling_4_size !=0 :

          model.add(MaxPooling2D(pool_size=pooling_4_size,padding='same'))

       model.add(Conv2D(filters = convlayer_5_filternum, kernel_size = convlayer_5_size, padding = 'same', activation = 'relu'))

       if pooling_5_size !=0 :

          model.add(MaxPooling2D(pool_size=pooling_5_size,padding='same'))

       model.add(Conv2D(filters = convlayer_6_filternum, kernel_size = convlayer_6_size, padding = 'same', activation = 'relu'))

       if pooling_6_size !=0 :

          model.add(MaxPooling2D(pool_size=pooling_6_size,padding='same'))

       model.add(Conv2D(filters = convlayer_7_filternum, kernel_size = convlayer_7_size, padding = 'same', activation = 'relu'))

       if pooling_7_size !=0 :

          model.add(MaxPooling2D(pool_size=pooling_7_size,padding='same'))

       model.add(Conv2D(filters = convlayer_8_filternum, kernel_size = convlayer_8_size, padding = 'same', activation = 'relu'))

       if pooling_8_size !=0 :

          model.add(MaxPooling2D(pool_size=pooling_8_size,padding='same'))

       model.add(Conv2D(filters = convlayer_9_filternum, kernel_size = convlayer_9_size, padding = 'same', activation = 'relu'))

       if pooling_9_size !=0 :

          model.add(MaxPooling2D(pool_size=pooling_9_size,padding='same'))

       model.add(Conv2D(filters = convlayer_10_filternum, kernel_size = convlayer_10_size, padding = 'same', activation = 'relu'))

       if pooling_10_size !=0 :

          model.add(MaxPooling2D(pool_size=pooling_10_size,padding='same'))

     #11 layer
    elif  convlayer_1_filternum !=0 and convlayer_2_filternum !=0 and convlayer_3_filternum!=0 and  convlayer_4_filternum!=0 and  convlayer_5_filternum!=0 and convlayer_6_filternum !=0 and convlayer_7_filternum !=0 and convlayer_8_filternum !=0 and convlayer_9_filternum !=0 and convlayer_10_filternum !=0 and convlayer_11_filternum!=0  and convlayer_12_filternum ==0  :

       print("to black 11");
       model.add(Conv2D(filters = convlayer_1_filternum, kernel_size = convlayer_1_size, padding = 'same', activation = 'relu', input_shape = (inputdim,inputdim,3)))

       if pooling_1_size !=0 :

          model.add(MaxPooling2D(pool_size=pooling_1_size,padding='same'))


       model.add(Conv2D(filters = convlayer_2_filternum, kernel_size = convlayer_2_size, padding = 'same', activation = 'relu'))

       if pooling_2_size !=0 :

          model.add(MaxPooling2D(pool_size=pooling_2_size,padding='same'))



       model.add(Conv2D(filters = convlayer_3_filternum, kernel_size = convlayer_3_size, padding = 'same', activation = 'relu'))

       if pooling_3_size !=0 :

          model.add(MaxPooling2D(pool_size=pooling_3_size,padding='same'))


       model.add(Conv2D(filters = convlayer_4_filternum, kernel_size = convlayer_4_size, padding = 'same', activation = 'relu'))

       if pooling_4_size !=0 :

          model.add(MaxPooling2D(pool_size=pooling_4_size,padding='same'))

       model.add(Conv2D(filters = convlayer_5_filternum, kernel_size = convlayer_5_size, padding = 'same', activation = 'relu'))

       if pooling_5_size !=0 :

          model.add(MaxPooling2D(pool_size=pooling_5_size,padding='same'))

       model.add(Conv2D(filters = convlayer_6_filternum, kernel_size = convlayer_6_size, padding = 'same', activation = 'relu'))

       if pooling_6_size !=0 :

          model.add(MaxPooling2D(pool_size=pooling_6_size,padding='same'))

       model.add(Conv2D(filters = convlayer_7_filternum, kernel_size = convlayer_7_size, padding = 'same', activation = 'relu'))

       if pooling_7_size !=0 :

          model.add(MaxPooling2D(pool_size=pooling_7_size,padding='same'))

       model.add(Conv2D(filters = convlayer_8_filternum, kernel_size = convlayer_8_size, padding = 'same', activation = 'relu'))

       if pooling_8_size !=0 :

          model.add(MaxPooling2D(pool_size=pooling_8_size,padding='same'))

       model.add(Conv2D(filters = convlayer_9_filternum, kernel_size = convlayer_9_size, padding = 'same', activation = 'relu'))

       if pooling_9_size !=0 :

          model.add(MaxPooling2D(pool_size=pooling_9_size,padding='same'))


       model.add(Conv2D(filters = convlayer_10_filternum, kernel_size = convlayer_10_size, padding = 'same', activation = 'relu'))

       if pooling_10_size !=0 :

          model.add(MaxPooling2D(pool_size=pooling_10_size,padding='same'))


       model.add(Conv2D(filters = convlayer_11_filternum, kernel_size = convlayer_11_size, padding = 'same', activation = 'relu'))

       if pooling_11_size !=0 :

          model.add(MaxPooling2D(pool_size=pooling_11_size,padding='same'))


     #12 layer
    elif  convlayer_1_filternum !=0 and convlayer_2_filternum !=0 and convlayer_3_filternum!=0 and  convlayer_4_filternum!=0 and  convlayer_5_filternum!=0 and convlayer_6_filternum !=0 and convlayer_7_filternum !=0 and convlayer_8_filternum !=0 and convlayer_9_filternum !=0 and convlayer_10_filternum !=0 and convlayer_11_filternum!=0  and convlayer_12_filternum !=0 and convlayer_13_filternum==0  :

       print("to black 12");
       model.add(Conv2D(filters = convlayer_1_filternum, kernel_size = convlayer_1_size, padding = 'same', activation = 'relu', input_shape = (inputdim,inputdim,3)))

       if pooling_1_size !=0 :

          model.add(MaxPooling2D(pool_size=pooling_1_size,padding='same'))


       model.add(Conv2D(filters = convlayer_2_filternum, kernel_size = convlayer_2_size, padding = 'same', activation = 'relu'))

       if pooling_2_size !=0 :

          model.add(MaxPooling2D(pool_size=pooling_2_size,padding='same'))



       model.add(Conv2D(filters = convlayer_3_filternum, kernel_size = convlayer_3_size, padding = 'same', activation = 'relu'))

       if pooling_3_size !=0 :

          model.add(MaxPooling2D(pool_size=pooling_3_size,padding='same'))


       model.add(Conv2D(filters = convlayer_4_filternum, kernel_size = convlayer_4_size, padding = 'same', activation = 'relu'))

       if pooling_4_size !=0 :

          model.add(MaxPooling2D(pool_size=pooling_4_size,padding='same'))

       model.add(Conv2D(filters = convlayer_5_filternum, kernel_size = convlayer_5_size, padding = 'same', activation = 'relu'))

       if pooling_5_size !=0 :

          model.add(MaxPooling2D(pool_size=pooling_5_size,padding='same'))

       model.add(Conv2D(filters = convlayer_6_filternum, kernel_size = convlayer_6_size, padding = 'same', activation = 'relu'))

       if pooling_6_size !=0 :

          model.add(MaxPooling2D(pool_size=pooling_6_size,padding='same'))

       model.add(Conv2D(filters = convlayer_7_filternum, kernel_size = convlayer_7_size, padding = 'same', activation = 'relu'))

       if pooling_7_size !=0 :

          model.add(MaxPooling2D(pool_size=pooling_7_size,padding='same'))

       model.add(Conv2D(filters = convlayer_8_filternum, kernel_size = convlayer_8_size, padding = 'same', activation = 'relu'))

       if pooling_8_size !=0 :

          model.add(MaxPooling2D(pool_size=pooling_8_size,padding='same'))

       model.add(Conv2D(filters = convlayer_9_filternum, kernel_size = convlayer_9_size, padding = 'same', activation = 'relu'))

       if pooling_9_size !=0 :

          model.add(MaxPooling2D(pool_size=pooling_9_size,padding='same'))


       model.add(Conv2D(filters = convlayer_10_filternum, kernel_size = convlayer_10_size, padding = 'same', activation = 'relu'))

       if pooling_10_size !=0 :

          model.add(MaxPooling2D(pool_size=pooling_10_size,padding='same'))


       model.add(Conv2D(filters = convlayer_11_filternum, kernel_size = convlayer_11_size, padding = 'same', activation = 'relu'))

       if pooling_11_size !=0 :

          model.add(MaxPooling2D(pool_size=pooling_11_size,padding='same'))


       model.add(Conv2D(filters = convlayer_12_filternum, kernel_size = convlayer_12_size, padding = 'same', activation = 'relu'))

       if pooling_12_size !=0 :

          model.add(MaxPooling2D(pool_size=pooling_12_size,padding='same'))

     #13 layer
    elif  convlayer_1_filternum !=0 and convlayer_2_filternum !=0 and convlayer_3_filternum!=0 and  convlayer_4_filternum!=0 and  convlayer_5_filternum!=0 and convlayer_6_filternum !=0 and convlayer_7_filternum !=0 and convlayer_8_filternum !=0 and convlayer_9_filternum !=0 and convlayer_10_filternum !=0 and convlayer_11_filternum!=0  and convlayer_12_filternum !=0 and convlayer_13_filternum!=0  :

       print("to black 13");
       model.add(Conv2D(filters = convlayer_1_filternum, kernel_size = convlayer_1_size, padding = 'same', activation = 'relu', input_shape = (inputdim,inputdim,3)))

       if pooling_1_size !=0 :

          model.add(MaxPooling2D(pool_size=pooling_1_size,padding='same'))


       model.add(Conv2D(filters = convlayer_2_filternum, kernel_size = convlayer_2_size, padding = 'same', activation = 'relu'))

       if pooling_2_size !=0 :

          model.add(MaxPooling2D(pool_size=pooling_2_size,padding='same'))



       model.add(Conv2D(filters = convlayer_3_filternum, kernel_size = convlayer_3_size, padding = 'same', activation = 'relu'))

       if pooling_3_size !=0 :

          model.add(MaxPooling2D(pool_size=pooling_3_size,padding='same'))


       model.add(Conv2D(filters = convlayer_4_filternum, kernel_size = convlayer_4_size, padding = 'same', activation = 'relu'))

       if pooling_4_size !=0 :

          model.add(MaxPooling2D(pool_size=pooling_4_size,padding='same'))

       model.add(Conv2D(filters = convlayer_5_filternum, kernel_size = convlayer_5_size, padding = 'same', activation = 'relu'))

       if pooling_5_size !=0 :

          model.add(MaxPooling2D(pool_size=pooling_5_size,padding='same'))

       model.add(Conv2D(filters = convlayer_6_filternum, kernel_size = convlayer_6_size, padding = 'same', activation = 'relu'))

       if pooling_6_size !=0 :

          model.add(MaxPooling2D(pool_size=pooling_6_size,padding='same'))

       model.add(Conv2D(filters = convlayer_7_filternum, kernel_size = convlayer_7_size, padding = 'same', activation = 'relu'))

       if pooling_7_size !=0 :

          model.add(MaxPooling2D(pool_size=pooling_7_size,padding='same'))

       model.add(Conv2D(filters = convlayer_8_filternum, kernel_size = convlayer_8_size, padding = 'same', activation = 'relu'))

       if pooling_8_size !=0 :

          model.add(MaxPooling2D(pool_size=pooling_8_size,padding='same'))

       model.add(Conv2D(filters = convlayer_9_filternum, kernel_size = convlayer_9_size, padding = 'same', activation = 'relu'))

       if pooling_9_size !=0 :

          model.add(MaxPooling2D(pool_size=pooling_9_size,padding='same'))


       model.add(Conv2D(filters = convlayer_10_filternum, kernel_size = convlayer_10_size, padding = 'same', activation = 'relu'))

       if pooling_10_size !=0 :

          model.add(MaxPooling2D(pool_size=pooling_10_size,padding='same'))


       model.add(Conv2D(filters = convlayer_11_filternum, kernel_size = convlayer_11_size, padding = 'same', activation = 'relu'))

       if pooling_11_size !=0 :

          model.add(MaxPooling2D(pool_size=pooling_11_size,padding='same'))


       model.add(Conv2D(filters = convlayer_12_filternum, kernel_size = convlayer_12_size, padding = 'same', activation = 'relu'))

       if pooling_12_size !=0 :

          model.add(MaxPooling2D(pool_size=pooling_12_size,padding='same'))


       model.add(Conv2D(filters = convlayer_13_filternum, kernel_size = convlayer_13_size, padding = 'same', activation = 'relu'))

       if pooling_13_size !=0 :

          model.add(MaxPooling2D(pool_size=pooling_13_size,padding='same'))


    model.add(Flatten())


    if dense_1_nerouns !=0 and dense_2_nerouns ==0:

        model.add(Dense(dense_1_nerouns, activation='relu'))

        model.add(Dropout(0.5))

        model.add(Dense(7, activation='softmax',name='outputlayer'))


    if dense_1_nerouns !=0 and dense_2_nerouns !=0 and dense_3_nerouns == 0 :

        model.add(Dense(dense_1_nerouns, activation='relu'))

        model.add(Dense(dense_2_nerouns, activation='relu'))

        model.add(Dropout(0.5))

        model.add(Dense(7, activation='softmax',name='outputlayer'))


    if dense_1_nerouns !=0 and dense_2_nerouns !=0 and dense_3_nerouns != 0:

        model.add(Dense(dense_1_nerouns, activation='relu'))

        model.add(Dense(dense_2_nerouns, activation='relu'))

        model.add(Dense(dense_3_nerouns, activation='relu'))

        model.add(Dropout(0.5))

        model.add(Dense(7, activation='softmax',name='outputlayer'))




    return(model)



population=[]



def random_initial() :

    invidual=[]

    #convlayer_numbes 0

    numlayers=random.randint(2,13)

    invidual.append(numlayers)

    for i in range(numlayers):

       #numfirst_convfilter  1
       invidual.append(random.choice(numconvfilterslist))

       #numfirst_convfiltersize 2
       invidual.append(random.choice(numconvfiltersizelist))

       #numfirst_convpoolingsize 3
       invidual.append(random.choice(numpoolingsizelist))

    for j in range(13-numlayers):

       #numfirst_convfilter  1
       invidual.append(0)

       #numfirst_convfiltersize 2
       invidual.append(0)

       #numfirst_convpoolingsize 3
       invidual.append(0)


     #dense/layers_numbes 13
    denselayer=random.randint(1,3)
    invidual.append(denselayer)

    if denselayer==1:

        #numfirst_convfiltersize 14
        invidual.append(random.choice(num_dense_nerons_list))

        #numfirst_convfiltersize 15

        invidual.append(0)

        #numfirst_convfiltersize 16
        invidual.append(0)

    if denselayer==2:

        #numfirst_convfiltersize 14
        invidual.append(random.choice(num_dense_nerons_list))

        #numfirst_convfiltersize 15

        invidual.append(random.choice(num_dense_nerons_list))

        #numfirst_convfiltersize 16
        invidual.append(0)

    if denselayer==3:

        #numfirst_convfiltersize 14
        invidual.append(random.choice(num_dense_nerons_list))

        #numfirst_convfiltersize 15

        invidual.append(random.choice(num_dense_nerons_list))

        #numfirst_convfiltersize 16
        invidual.append(random.choice(num_dense_nerons_list))

    return invidual

def CrossOver(parents, offspring_size):

     offspring = numpy.empty((10,44))

     offspring=offspring.astype("int32")

     crossover_point = numpy.int32(22)

     for k in range(10):

         # Index of the first parent to mate.
         parent1_idx = k % parents.shape[0]

         #print('parent1_idx==>',parent1_idx)

         #print(parents[parent1_idx, 0:crossover_point])
         # Index of the second parent to mate.
         parent2_idx = (k+1) % parents.shape[0]

         #print('parent2_idx==>',parent2_idx)

         #print(parents[parent2_idx, crossover_point:])

         # The new offspring will have its first half of its genes taken from the first parent.
         offspring[k, 0:crossover_point] = parents[parent1_idx, 0:crossover_point]

         # The new offspring will have its second half of its genes taken from the second parent.
         offspring[k, crossover_point:] = parents[parent2_idx, crossover_point:]

     return offspring


def Mutation(offspring_crossover):

    # Mutation changes a single gene in each offspring randomly.

    for idx in range(offspring_crossover.shape[0]):

        index=random.randint(0,43)

        randomvalue=0

        if index==0:


           index=index+1


        if index==1 or index==4 or index==7 or index==10 or index==13 or index==16 or index==19 or index==22 or index==25 or index==28 or index==31 or index==34 or index==37:

            if offspring_crossover[idx, index+1] == 0 :

                randomvalue=0

                index=index+1

            else:

               randomvalue=random.choice(numconvfilterslist)



        if index==2 or index==5 or index==8 or index==11 or index==14 or index==17 or index==20 or index==23 or index==26 or index==29 or index==32 or index==35 or index==38 :

            randomvalue=random.choice(numconvfiltersizelist)


        if index==3 or index==6 or index==9 or index==12 or index==15 or index==18 or index==21 or index==24 or index==27 or index==30 or index==33 or index==36 or index==39 :

            randomvalue=random.choice(numpoolingsizelist)


        if index==40:

           index=index+1


        if index==41 or index==42 or index==43:

           randomvalue=random.choice(num_dense_nerons_list)

        offspring_crossover[idx, index] =randomvalue


    return offspring_crossover


def CallFitness(invidual,modelname):



    model=createDeeepModel(invidual[0],
                           invidual[1],invidual[2],invidual[3],
                           invidual[4],invidual[5],invidual[6],
                           invidual[7],invidual[8],invidual[9],
                           invidual[10],invidual[11],invidual[12],
                           invidual[13],invidual[14],invidual[15],
                           invidual[16],invidual[17],invidual[18],
                           invidual[19],invidual[20],invidual[21],

                           invidual[22],invidual[23],invidual[24],
                           invidual[25],invidual[26],invidual[27],
                           invidual[28],invidual[29],invidual[30],
                           invidual[31],invidual[32],invidual[33],
                           invidual[34],invidual[35],invidual[36],
                           invidual[37],invidual[38],invidual[39],

                           invidual[40],
                           invidual[41],invidual[42],invidual[43],
                           )



    model.compile(loss='SparseCategoricalCrossentropy', optimizer='adam', metrics=['accuracy'])

    #model.summary()

    callbacks = [EarlyStopping(monitor='val_accuracy', patience=3)]

    #model.fit_generator(train_generator.flow(X_train, y_train, batch_size=32),epochs=20,callbacks=callbacks,steps_per_epoch=len(X_train)//batch_size,validation_data=(X_test, y_test),verbose=2)

    model.fit(newtrainimages, newtrainlabels, epochs=40, batch_size=32,callbacks=callbacks,validation_data=(testimages, testlabels), verbose=2)

    scores= model.evaluate(testimages, testlabels)

    #int("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

    fitness=scores[1]*100

    #Trying to save model to the disk

    if fitness > 80 :

      #model.save(modelname+'.h5')

      print (modelname," is succefully saved !")

    del model

    gc.collect()

    print("checkpoint 3 for  ram ")

    return fitness


def PrintPopulation(Population):

    for idx in range(20):

        print('###########################################################')

        for index in range(18):


            if index==0:

               print('number of conv layer: ',Population[idx,index])


            if index==1 or index==4 or index==7 or index==10:

               print('number of convfilter ',Population[idx,index])


            if index==2 or index==5 or index==8 or index==11:

               print('filtersize is :',Population[idx,index])


            if index==3 or index==6 or index==9 or index==12:

               print('poolingsize is :',Population[idx,index])


            if index==13:

                print('number of dense layer: ',Population[idx,index])



            if index==14 or index==15 or index==16:

               print('number of nerouns is: ',Population[idx,index])







Runing of genetic algorithm for finding best architecture and hyper-parameter optimization of scratch models

In [ ]:

import copy

from keras.optimizers import Adam

fitness=[]


for i in range(0,20):

  invidual=random_initial()

  population.append(invidual)

  print('individual ',i,' is :')

  modelname="Generation_0_individual_"+str(i)

  print(invidual)


  fitvalue=CallFitness(invidual,modelname)


  print("model name is ===> ",modelname)

  print('fittness of individual',i,'is',fitvalue)

  fitness.append(fitvalue)

 # Selecting the best parents in the population for mating.





num_generations = 20



population=numpy.array( population).astype("int32")



for generation in range(num_generations):

     print("####################################### Generation : ", generation, "#############################")

     print("")

     #print(' population ===> ')

     #print(population)

     fitness2=copy.deepcopy(fitness)

     #befor enter here copy value and not refrence

     temp_array = findmaximum(copy.deepcopy(fitness),10)

     parents=select_mating_pool(numpy.array(population).astype("int32"),fitness,10)

     # Generating next generation using crossover

     offspring_crossover = CrossOver(parents,offspring_size=(20-parents.shape[0]))

     #print("Crossover")

     #print(offspring_crossover)

     # Adding some variations to the offsrping using mutation.



     offspring_mutation = Mutation(offspring_crossover)


     # Creating the new population based on the parents and offspring.
     population[0:10 , :] = parents



     population[10:, :] = offspring_mutation



     fitness=[]

     #add 10 most fittest prior parents to next stage

     for i in range(10):

       fitness.append(fitness2[ temp_array[i]])





     for i in range(10,20):



         p=population[i].tolist()

         print("individual ",i);

         modelname="Generation_"+ str(generation)+ "_individual_"+ str(i)

         print(p);

         fitvalue=CallFitness(p,modelname)

         print("model name is ===> ",modelname)

         print('fittness of round',i,'is',fitvalue)

         fitness.append(fitvalue)

     print('fitness List of generation ',generation)

     print(fitness)

